In [1]:
# !pip install transformers==4.41.2 sentence-transformers==3.3.0 datasets evaluate accelerate --upgrade --quiet
!pip install git+https://github.com/huggingface/transformers.git --upgrade --quiet
!pip install -U accelerate --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 840.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.0 MB/s eta 0:00:00


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd
import torch

In [5]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv("Final_ZC_Admission_Dataset .csv")

# Format it for T5
df["input_text"] = "question: " + df["question"]
df["target_text"] = df["answer"]
dataset = Dataset.from_pandas(df)

In [6]:
df.head()

,question,answer,input_text,target_text
0,What are you?,I am an LLM model developed because some stude...,question: What are you?,I am an LLM model developed because some stude...
1,Are you a good AI or a bad AI,Currently I am still in Alpha version! lets me...,question: Are you a good AI or a bad AI,Currently I am still in Alpha version! lets me...
2,Rema? who picked this name,A passionate student,question: Rema? who picked this name,A passionate student
3,Hey,"Hey, How can I be of service?",question: Hey,"Hey, How can I be of service?"
4,Your eyes are beautiful,"Thanks, How can I be of service?",question: Your eyes are beautiful,"Thanks, How can I be of service?"


In [7]:
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess(example):
    inputs = tokenizer(example["input_text"], padding="max_length", truncation=True, max_length=64)
    targets = tokenizer(example["target_text"], padding="max_length", truncation=True, max_length=32)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True)
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/71 [00:00<?, ? examples/s]

In [8]:
import transformers
print(transformers.__version__)

4.52.0.dev0


In [9]:
!pip install bitsandbytes datasets accelerate loralib

!pip install git+https://github.com/huggingface/peft.git

!pip install git+https://github.com/huggingface/transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.6 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-xi7m28jh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-xi7m28jh
  Resolved https://github.com/huggingface/peft.git to commit 6c489499300c652a4990cfbcc18539417e73c262
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.15.2.dev0-py3-none-any.whl size=439324 sha256=483ed8855feff6be81a63f7fbb84e1901cf7997335c390099f5c804d630ac8ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-vjxan4zl/wheels/42/ec/c4/eb24dac74be83ba2ed4817037a784d1c775e317cb8de69963f
Successfully built peft
  Attempting uninstall: peft
    Found existing installation: peft 0.15.2
    Uninstalling peft-0.15.2:
      Successfully uninstalled peft-0.15.2


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-f_9bil51
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-f_9bil51
  Resolved https://github.com/huggingface/transformers to commit 716819b8309324302e00a3488a3c3d6faa427f79
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [10]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

training_args = TrainingArguments(
    output_dir="./t5-qa-model",
    per_device_train_batch_size=10,
    per_device_eval_batch_size=8,
    num_train_epochs=70,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=20,
    learning_rate=3e-4,
    weight_decay=0.01,
    warmup_steps=50,
    report_to="none",
    fp16=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer
)

<ipython-input-11-5942a52fb4d7>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,5.940495
2,No log,5.448967
3,6.170000,4.623564
4,6.170000,3.541119
5,6.170000,3.139662
6,4.074900,2.684501
7,4.074900,2.128845
8,4.074900,1.685656
9,2.580600,1.289616
10,2.580600,0.958491


TrainOutput(global_step=490, training_loss=0.6466340648428518, metrics={'train_runtime': 1629.7026, 'train_samples_per_second': 2.706, 'train_steps_per_second': 0.301, 'total_flos': 74607168061440.0, 'train_loss': 0.6466340648428518, 'epoch': 70.0})

In [13]:
model.save_pretrained("my_model")
tokenizer.save_pretrained("my_model")

('my_model/tokenizer_config.json',
 'my_model/special_tokens_map.json',
 'my_model/spiece.model',
 'my_model/added_tokens.json')

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("my_qa_model")
model = T5ForConditionalGeneration.from_pretrained("my_qa_model")

def answer_question(question):
    input_text = "question: " + question
    inputs = tokenizer(input_text, return_tensors="pt")
    output = model.generate(**inputs, max_length=4000)
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return "\n".join(sentence.strip() for sentence in response.split(".") if sentence)

# Test
q = ''
while q != 'bye':
  q = input("You: ")
  print("Rema: ", answer_question(q))